<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/notebooks/ForceAlighment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
!gdown 1MrkteyPMIhLgki82_qJ_A9JDOIme3za3
!unzip -q 100-samples.zip

Downloading...
From (original): https://drive.google.com/uc?id=1MrkteyPMIhLgki82_qJ_A9JDOIme3za3
From (redirected): https://drive.google.com/uc?id=1MrkteyPMIhLgki82_qJ_A9JDOIme3za3&confirm=t&uuid=6d9fa167-59e7-4da5-a59f-45896c6c0756
To: /kaggle/working/100-samples.zip
100%|██████████████████████████████████████| 52.8M/52.8M [00:01<00:00, 40.7MB/s]


In [ ]:
!gdown 121dcbCvDVzB22uelCeYkg4Fs8TVbQ1tp

Downloading...
From: https://drive.google.com/uc?id=121dcbCvDVzB22uelCeYkg4Fs8TVbQ1tp
To: /kaggle/working/100-samples.csv
100%|██████████████████████████████████████| 29.0k/29.0k [00:00<00:00, 54.2MB/s]


In [ ]:
import pandas as pd

data = pd.read_csv("100-samples.csv")

In [ ]:
data.head()

,Unnamed: 0,id,type,command,sampling_rate,num_segments,type_segments,text_segments,ref_id,ref_file,ref_text
0,0,5e8907d2,non_active,I remember watching so many episodes of Nickel...,24000,0,{},{},EN_B00013_S02252_W000000,/content/audio/dung/EN_B00013_S02712_W000000.wav,/content/json/dung/EN_B00013_S02252_W000000.json
1,1,4ff7f40d,single_active,show me Cartoon Network,24000,0,{},{},EN_B00013_S02242_W000000,/content/audio/dung/EN_B00013_S02462_W000000.wav,/content/json/dung/EN_B00013_S02242_W000000.json
2,2,d337321c,single_mix,Can you enable game mode for today's match bec...,24000,3,"{'0': 'non_active', '1': 'active', '2': 'non_a...","{'0': 'Can you', '1': 'enable game mode', '2':...",EN_B00013_S02592_W000000,/content/audio/dung/EN_B00013_S02712_W000000.wav,/content/json/dung/EN_B00013_S02592_W000000.json
3,3,af7b2342,chain_active,"increase the volume, switch to Cartoon Network",24000,0,{},{},EN_B00013_S02242_W000000,/content/audio/dung/EN_B00013_S02672_W000000.wav,/content/json/dung/EN_B00013_S02242_W000000.json
4,4,79b56df5,chain_mix,"set volume to 10, do you remember my birthday ...",24000,3,"{'0': 'active', '1': 'non_active', '2': 'active'}","{'0': 'set volume to 10', '1': 'do you remembe...",EN_B00013_S02602_W000000,/content/audio/dung/EN_B00013_S02552_W000000.wav,/content/json/dung/EN_B00013_S02602_W000000.json


# Model

In [ ]:
!pip install -q -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.

In [ ]:
import whisper
from pydub import AudioSegment

model = whisper.load_model("medium")

# Pipeline

## Cleaning

### Function Preparation

In [ ]:
import numpy as np
import re
from difflib import SequenceMatcher

import ast

def normalize_text(text):
    return re.sub(r'[^\w\s]', '', text.lower()).strip()

def word_similarity(w1, w2):
    return SequenceMatcher(None, w1, w2).ratio()

def smith_waterman_fuzzy(ref_words, hypo_words, match_score=2, fuzzy_score=1, mismatch=-1, gap=-2):
    m, n = len(ref_words), len(hypo_words)
    score = np.zeros((m+1, n+1))
    max_score = 0
    max_pos = None

    for i in range(1, m+1):
        for j in range(1, n+1):
            sim = word_similarity(ref_words[i-1], hypo_words[j-1])
            if sim == 1:
                s = match_score
            elif sim >= 0.8:
                s = fuzzy_score
            else:
                s = mismatch

            diag = score[i-1, j-1] + s
            delete = score[i-1, j] + gap
            insert = score[i, j-1] + gap
            score[i, j] = max(0, diag, delete, insert)

            if score[i, j] > max_score:
                max_score = score[i, j]
                max_pos = (i, j)

    # Traceback
    i, j = max_pos
    end_j = j - 1
    while i > 0 and j > 0 and score[i, j] > 0:
        sim = word_similarity(ref_words[i-1], hypo_words[j-1])
        if sim >= 0.8:
            i -= 1
            j -= 1
        elif score[i-1, j] + gap == score[i, j]:
            i -= 1
        else:
            j -= 1
    start_j = j

    return start_j, end_j

def get_start_end_from_alignment(ref_text, whisper_words):
    ref_words = normalize_text(ref_text).split()
    hypo_words = [normalize_text(w['word']) for w in whisper_words]

    start_idx, end_idx = smith_waterman_fuzzy(ref_words, hypo_words)
    start_time = whisper_words[start_idx]['start']
    end_time = whisper_words[end_idx]['end']

    return start_time, end_time


def str2list(list_str: str):
    return ast.literal_eval(list_str)

def trim_audio(file_name, file_type, start, end, data_folder = "synthesis_command"):
    file_path = os.path.join(data_folder, file_name, f"{file_name}_{file_type}.wav")

    audio = AudioSegment.from_wav(file_path)
    if start and end:

        trimmed_audio = audio[start*1000 : end*1000]
        return trimmed_audio
    else:
        return audio

## Cleaning Pipeline

In [ ]:
cant_clean_list = []

In [ ]:
cant_clean_list

[('single_active_4ff7f40d', 'full', 'cant get start end'),
 ('single_mix_d337321c', 'seg_0', 'cant get start end'),
 ('chain_mix_79b56df5', 'full', 'cant get start end'),
 ('single_mix_5143fc18', 'seg_3', 'cant get start end'),
 ('single_mix_5143fc18', 'seg_9', 'cant get start end'),
 ('single_mix_5143fc18', 'seg_10', 'cant get start end'),
 ('single_active_7aa90624', 'full', 'cant get start end'),
 ('single_mix_c6252315', 'seg_0', 'cant get start end'),
 ('single_mix_c6252315', 'seg_1', 'cant get start end'),
 ('single_mix_c6252315', 'seg_2', 'cant get start end'),
 ('single_mix_c6252315', 'seg_3', 'cant get start end'),
 ('single_mix_c6252315', 'seg_4', 'cant get start end'),
 ('single_mix_c6252315', 'seg_5', 'cant get start end'),
 ('single_mix_c6252315', 'seg_7', 'cant get start end'),
 ('single_mix_c6252315', 'seg_8', 'cant get start end'),
 ('single_mix_c6252315', 'seg_9', 'cant get start end'),
 ('single_mix_c6252315', 'seg_10', 'cant get start end'),
 ('chain_mix_0969edce', 'se

In [ ]:
import json
import os
import itertools

def get_clean_range(file_name: str, file_type: str, data_folder: str = "synthesis_command", padding = None):
    # prepare whisper words
    file_path = os.path.join(data_folder, file_name, f"{file_name}_{file_type}.wav")

    transcribe = model.transcribe(file_path, word_timestamps=True)
    try:
        whisper_words = [speaker['words'] for speaker in transcribe['segments']]

        whisper_words = list(itertools.chain.from_iterable(whisper_words))
    except Exception as e:
        print(file_name + " " + file_type + " " + 'cant transcribe')
        cant_clean_list.append((file_name, file_type, "cant transcribe"))
        return None, None

    # prepare reference text
    meta_path = os.path.join(data_folder, file_name, f"{file_name}.json")

    with open(meta_path, "r", encoding = "utf-8") as f:
        data = json.load(f)

    if file_type == "full":
        ref_text = data['command']
    else:
        num_seg = file_type.split("_")[1]
        ref_text = data['text_segments'][num_seg]

    try:
        start, end = get_start_end_from_alignment(ref_text = ref_text,
                                              whisper_words = whisper_words)
        if padding:
            start = max(0, start - padding)
            end = end + padding
    except:
        # Problems with long synthesis, should use only with segment
        print(file_name + " " + file_type + " " + 'cant get start end')
        cant_clean_list.append((file_name, file_type, "cant get start end"))
        return None, None
    return start, end

In [ ]:
def clean_pipeline(export_dir: str, data: pd.DataFrame, speech_dir: str = "synthesis_command", padding = None):
    os.makedirs(export_dir, exist_ok = True)
    for idx, row in data.iterrows():

        file_name = f"{row['type']}_{row['id']}"
        file_type = "full"

        start, end = get_clean_range(file_name = file_name, file_type = file_type, padding = padding)
        trimmed_audio = trim_audio(file_name = file_name, file_type = file_type,
                                  start = start, end = end)

        os.makedirs(os.path.join(export_dir, file_name), exist_ok = True)
        out_path = os.path.join(export_dir, file_name, f"{file_name}_{file_type}_trimmed.wav")
        trimmed_audio.export(out_path, format="wav")

        if row['type'] in ['single_mix', 'chain_mix']:

           for i in range(row['num_segments']):
               file_type = f"seg_{i}"
               start, end = get_clean_range(file_name = file_name, file_type = file_type, padding = padding)
               trimmed_audio = trim_audio(file_name = file_name, file_type = file_type,
                                      start = start, end = end)

               os.makedirs(os.path.join(export_dir, file_name), exist_ok = True)
               out_path = os.path.join(export_dir, file_name, f"{file_name}_{file_type}_trimmed.wav")
               trimmed_audio.export(out_path, format="wav")

In [ ]:
df = pd.read_csv("/kaggle/working/100-samples.csv", index_col = 0)

clean_pipeline(export_dir = 'trimmed_speech', data = df, padding = 0.3)

single_active_4ff7f40d full cant get start end
single_mix_d337321c seg_0 cant get start end
chain_mix_79b56df5 full cant get start end
single_mix_5143fc18 seg_3 cant get start end
single_mix_5143fc18 seg_9 cant get start end
single_mix_5143fc18 seg_10 cant get start end
single_active_7aa90624 full cant get start end
single_mix_c6252315 seg_0 cant get start end
single_mix_c6252315 seg_1 cant get start end
single_mix_c6252315 seg_2 cant get start end
single_mix_c6252315 seg_3 cant get start end
single_mix_c6252315 seg_4 cant get start end
single_mix_c6252315 seg_5 cant get start end
single_mix_c6252315 seg_7 cant get start end
single_mix_c6252315 seg_8 cant get start end
single_mix_c6252315 seg_9 cant get start end
single_mix_c6252315 seg_10 cant get start end
chain_mix_0969edce seg_0 cant get start end
chain_mix_460f79b4 seg_0 cant get start end
chain_mix_460f79b4 seg_1 cant get start end
single_mix_79892fde seg_2 cant get start end
chain_mix_b211eaf6 seg_2 cant get start end
single_act

In [ ]:
cant_clean_list

[('single_active_4ff7f40d', 'full', 'cant get start end'),
 ('single_mix_d337321c', 'seg_0', 'cant get start end'),
 ('chain_mix_79b56df5', 'full', 'cant get start end'),
 ('single_mix_5143fc18', 'seg_3', 'cant get start end'),
 ('single_mix_5143fc18', 'seg_9', 'cant get start end'),
 ('single_mix_5143fc18', 'seg_10', 'cant get start end'),
 ('single_active_7aa90624', 'full', 'cant get start end'),
 ('single_mix_c6252315', 'seg_0', 'cant get start end'),
 ('single_mix_c6252315', 'seg_1', 'cant get start end'),
 ('single_mix_c6252315', 'seg_2', 'cant get start end'),
 ('single_mix_c6252315', 'seg_3', 'cant get start end'),
 ('single_mix_c6252315', 'seg_4', 'cant get start end'),
 ('single_mix_c6252315', 'seg_5', 'cant get start end'),
 ('single_mix_c6252315', 'seg_7', 'cant get start end'),
 ('single_mix_c6252315', 'seg_8', 'cant get start end'),
 ('single_mix_c6252315', 'seg_9', 'cant get start end'),
 ('single_mix_c6252315', 'seg_10', 'cant get start end'),
 ('chain_mix_0969edce', 'se

In [ ]:
df = pd.DataFrame(cant_clean_list, columns=["id", "segment", "error_message"])

# Xuất ra file CSV
df.to_csv("output.csv", index=False)

print("Đã lưu thành output.csv")

Đã lưu thành output.csv


In [ ]:
!zip trimmed_speech.zip trimmed_speech -r

updating: trimmed_speech/ (stored 0%)
  adding: trimmed_speech/chain_mix_0969edce/ (stored 0%)
  adding: trimmed_speech/chain_mix_0969edce/chain_mix_0969edce_seg_1_trimmed.wav (deflated 22%)
  adding: trimmed_speech/chain_mix_0969edce/chain_mix_0969edce_full_trimmed.wav (deflated 11%)
  adding: trimmed_speech/chain_mix_0969edce/chain_mix_0969edce_seg_0_trimmed.wav (deflated 49%)
  adding: trimmed_speech/chain_mix_0969edce/chain_mix_0969edce_seg_2_trimmed.wav (deflated 28%)
  adding: trimmed_speech/single_mix_c6252315/ (stored 0%)
  adding: trimmed_speech/single_mix_c6252315/single_mix_c6252315_seg_0_trimmed.wav (deflated 58%)
  adding: trimmed_speech/single_mix_c6252315/single_mix_c6252315_seg_2_trimmed.wav (deflated 62%)
  adding: trimmed_speech/single_mix_c6252315/single_mix_c6252315_full_trimmed.wav (deflated 10%)
  adding: trimmed_speech/single_mix_c6252315/single_mix_c6252315_seg_5_trimmed.wav (deflated 56%)
  adding: trimmed_speech/single_mix_c6252315/single_mix_c6252315_seg_10_t

# Experiment
- find best algo

In [ ]:
import pandas as pd

text_command = pd.read_csv("text_command.csv")

In [ ]:
choice = "000000020"

In [ ]:
import json

with open(f"synthesis_command/chain_mix_{choice}/chain_mix_{choice}.json", "r") as f:
  sample_json = json.load(f)

In [ ]:
import ast

def get_segment(text: str):
  return ast.literal_eval(text_command[text_command["text"] == text]["segments"].values[0])

In [ ]:
list_segment = get_segment(sample_json['command'])

In [ ]:
import numpy as np
import re
from difflib import SequenceMatcher

def normalize_text(text):
    return re.sub(r'[^\w\s]', '', text.lower()).strip()

def word_similarity(w1, w2):
    return SequenceMatcher(None, w1, w2).ratio()

def smith_waterman_fuzzy(ref_words, hypo_words, match_score=2, fuzzy_score=1, mismatch=-1, gap=-2):
    m, n = len(ref_words), len(hypo_words)
    score = np.zeros((m+1, n+1))
    max_score = 0
    max_pos = None

    for i in range(1, m+1):
        for j in range(1, n+1):
            sim = word_similarity(ref_words[i-1], hypo_words[j-1])
            if sim == 1:
                s = match_score
            elif sim >= 0.8:
                s = fuzzy_score
            else:
                s = mismatch

            diag = score[i-1, j-1] + s
            delete = score[i-1, j] + gap
            insert = score[i, j-1] + gap
            score[i, j] = max(0, diag, delete, insert)

            if score[i, j] > max_score:
                max_score = score[i, j]
                max_pos = (i, j)

    # Traceback
    i, j = max_pos
    end_j = j - 1
    while i > 0 and j > 0 and score[i, j] > 0:
        sim = word_similarity(ref_words[i-1], hypo_words[j-1])
        if sim >= 0.8:
            i -= 1
            j -= 1
        elif score[i-1, j] + gap == score[i, j]:
            i -= 1
        else:
            j -= 1
    start_j = j

    return start_j, end_j

def get_start_end_from_alignment(ref_text, whisper_words, padding = None):
    ref_words = normalize_text(ref_text).split()
    hypo_words = [normalize_text(w['word']) for w in whisper_words]

    start_idx, end_idx = smith_waterman_fuzzy(ref_words, hypo_words)
    start_time = whisper_words[start_idx]['start']
    end_time = whisper_words[end_idx]['end']

    if padding:
        start = max(0, start_time - padding)
        end = end_time + padding
    return start_time, end_time


In [ ]:
import os
list_transcribe = []
for i in range(len(list_segment)):
  file_path = os.path.join(f"synthesis_command/chain_mix_{choice}", f"chain_mix_{choice}_seg_{i}.wav")
  transcribe = model.transcribe(file_path, word_timestamps=True)
  list_transcribe.append((list_segment[i][str(i)], transcribe))


In [ ]:
for ref_text, whisper_output in list_transcribe:  # data = list bạn đưa ở trên
    whisper_words = whisper_output['segments'][0]['words']
    start, end = get_start_end_from_alignment(ref_text, whisper_words)
    print(ref_text, "=>", start, "-", end)


decrease the volume to 5 => 3.42 - 4.62
while I tell you about my day, then => 3.82 - 6.74
restart the TV => 1.6600000000000001 - 2.54
when I'm done? => 2.2 - 2.66


In [ ]:
output_dir = "trimmed_audio"
os.makedirs(output_dir, exist_ok=True)

for i, item in enumerate(list_transcribe):
    print(item)
    ref_text = item[0]  # text chuẩn
    whisper_output = item[1]  # dict chứa 'segments'

    file_path = os.path.join(
        f"synthesis_command/chain_mix_{choice}",
        f"chain_mix_{choice}_seg_{i}.wav"
    )

    whisper_words = whisper_output['segments'][0]['words']

    # Tìm start/end chuẩn
    start_time, end_time = get_start_end_from_alignment(ref_text, whisper_words)


    padding = 0.5
    print(start_time)
    start_time = max(0, start_time - padding)
    end_time += padding
    print(start_time)
    # Load audio
    audio = AudioSegment.from_wav(file_path)

    # Cắt (ms)
    trimmed_audio = audio[start_time*1000 : end_time*1000]

    # Lưu file
    out_path = os.path.join(output_dir, f"trimmed_seg_{i}.wav")
    trimmed_audio.export(out_path, format="wav")

    # print(f"Saved: {out_path} ({start_time:.2f}s - {end_time:.2f}s)")


('decrease the volume to 5', {'text': ' JSON, decrease the volume to 5.', 'segments': [{'id': 0, 'seek': 0, 'start': 1.8199999999999994, 'end': 4.62, 'text': ' JSON, decrease the volume to 5.', 'tokens': [50364, 31828, 11, 11514, 264, 5523, 281, 1025, 13, 50614], 'temperature': 0.0, 'avg_logprob': -0.6382030573758212, 'compression_ratio': 0.8378378378378378, 'no_speech_prob': 0.4032731354236603, 'words': [{'word': ' JSON,', 'start': 1.8199999999999994, 'end': 2.34, 'probability': 0.5002037882804871}, {'word': ' decrease', 'start': 3.42, 'end': 3.68, 'probability': 0.9192915558815002}, {'word': ' the', 'start': 3.68, 'end': 3.88, 'probability': 0.9911214113235474}, {'word': ' volume', 'start': 3.88, 'end': 4.12, 'probability': 0.9814786911010742}, {'word': ' to', 'start': 4.12, 'end': 4.4, 'probability': 0.9969452023506165}, {'word': ' 5.', 'start': 4.4, 'end': 4.62, 'probability': 0.44756704568862915}]}], 'language': 'en'})
3.42
2.92
('while I tell you about my day, then', {'text': ' J

In [ ]:
import os
from IPython.display import Audio, display

output_dir = "trimmed_audio"

# Lặp qua toàn bộ file wav trong thư mục
for file_name in sorted(os.listdir(output_dir)):
    if file_name.lower().endswith(".wav"):
        file_path = os.path.join(output_dir, file_name)
        print(f"▶️ {file_name}")
        display(Audio(file_path))


▶️ trimmed_seg_0.wav


▶️ trimmed_seg_1.wav


▶️ trimmed_seg_2.wav


▶️ trimmed_seg_3.wav


In [ ]:
import os
from IPython.display import Audio, display

input_dir = f"synthesis_command/chain_mix_{choice}/"

# Lặp qua toàn bộ file wav trong thư mục
for file_name in sorted(os.listdir(input_dir)):
    if file_name.lower().endswith(".wav"):
        try:
            file_path = os.path.join(input_dir, file_name)
            print(file_path)
            print(f"▶️ {file_name}")
            display(Audio(file_path))
        except:
            continue

synthesis_command/chain_mix_000000020/chain_mix_000000020_full.wav
▶️ chain_mix_000000020_full.wav


synthesis_command/chain_mix_000000020/chain_mix_000000020_seg_0.wav
▶️ chain_mix_000000020_seg_0.wav


synthesis_command/chain_mix_000000020/chain_mix_000000020_seg_1.wav
▶️ chain_mix_000000020_seg_1.wav


synthesis_command/chain_mix_000000020/chain_mix_000000020_seg_2.wav
▶️ chain_mix_000000020_seg_2.wav


synthesis_command/chain_mix_000000020/chain_mix_000000020_seg_3.wav
▶️ chain_mix_000000020_seg_3.wav


In [ ]:
import os
import random
import numpy as np
from pydub import AudioSegment

# Thư mục chứa file WAV
input_dir = "trimmed_audio"
output_file = "concatenated_audio.wav"

# Hàm tạo silence theo phân bố exponential (ms)
def exponential_silence(scale=500, min_silence=100):
    """
    scale: lambda của phân bố exponential, đơn vị ms
    min_silence: khoảng lặng tối thiểu
    """
    silence_duration = np.random.exponential(scale=scale)
    return max(int(silence_duration), min_silence)

# Lấy danh sách file WAV và sort để có thứ tự ổn định
wav_files = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(".wav")])

# Khởi tạo audio rỗng
final_audio = AudioSegment.silent(duration=0)

# Nối từng file audio + silence
for i, file_name in enumerate(wav_files):
    file_path = os.path.join(input_dir, file_name)
    audio = AudioSegment.from_wav(file_path)
    final_audio += audio

    # Nếu chưa phải file cuối, chèn silence ngẫu nhiên
    if i < len(wav_files) - 1:
        # silence_ms = exponential_silence(scale=500)  # scale=500ms trung bình
        silence_ms = 0
        final_audio += AudioSegment.silent(duration=silence_ms)

# Xuất file cuối cùng
final_audio.export(output_file, format="wav")
print(f"✅ Đã concat {len(wav_files)} file thành {output_file}")


✅ Đã concat 4 file thành concatenated_audio.wav


In [ ]:
from IPython.display import Audio, display

def display_concatenated(file_path):
    """
    Hiển thị file audio concat cuối cùng
    """
    print(f"▶️ Playing concatenated audio: {file_path}")
    display(Audio(file_path))

# File concat
concat_file = "concatenated_audio.wav"

# Chạy
display_concatenated(concat_file)


▶️ Playing concatenated audio: concatenated_audio.wav
